In [1]:
import pandas as pd
import csv
import re
import string
import nltk
from nltk.corpus import words

In [2]:
#load decade_statistics.csv
decade_stats = pd.read_csv('decade_statistics.csv')

In [3]:
decade_stats

,decade,num_articles,date_range_start,date_range_end,avg_text_length,file_size_mb
0,1850s,211158,18510918,18591231,4859.158237,711.827241
1,1860s,343722,18600102,18691231,4339.515402,1093.269255
2,1870s,518909,18700101,18791231,3336.186563,1218.452102
3,1880s,705417,18800101,18891231,2767.497689,1343.687194
4,1890s,735949,18900101,18991231,3445.433662,1611.972227
5,1900s,848116,19000101,19091231,3517.085702,1934.784999
6,1910s,914466,19100101,19191231,3451.005522,2131.416859
7,1920s,877644,19200101,19291231,2862.179664,1735.508478
8,1930s,495824,19300101,19361231,2830.881650,910.525859


In [4]:
fifties = pd.read_parquet('data/nyt_1850s.parquet', engine='pyarrow')

In [5]:
print(fifties.head())
print(fifties.describe())
print(fifties.info())

   pub_date                            object_type  \
0  18571121                      Article | Feature   
1  18571109                                   News   
2  18580407                      Article | Feature   
3  18580327                      Article | Feature   
4  18590613  Correspondence | Letter to the Editor   

                                            raw_text  
0  Sltenlbont I). THE CATARACT BLOWN CP--FIvE PER...  
1  ebe W-p: CilR5, , XOtitiltbCY 9, 1 _ * X . . ....  
2  The Collims Steamers for Philadelphla. From th...  
3  LATEST INTELLIGENCE By Telegraph to the New-Yo...  
4  Clean Sireets. r tLe Editor of t.Ar Neur York ...  
        pub_date        object_type raw_text
count     211158             211158   211158
unique      2592                 15   211108
top     18521004  Article | Feature      ...
freq         150             108967       17
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211158 entries, 0 to 211157
Data columns (total 3 columns):
 #   Colum

In [6]:
#checks for every use of word utopia in the 1850s dataset
utopia_fifties = fifties[fifties['raw_text'].str.contains('utopia', case=False, na=False)]

In [7]:
#function that checks for number of occurrences of a word in the the entire dataset
def count_word_occurrences(dataset, word):
    pattern = re.compile(r'\b' + re.escape(word) + r'\b', re.IGNORECASE)
    return dataset['raw_text'].str.count(pattern).sum()

count_word_occurrences(fifties, "utopian")


56

In [8]:
utopia_fifties

,pub_date,object_type,raw_text
3030,18560319,Advertisement | Classified Advertisement,"Jo-l m - -Aboi 1Is to 20 ya.ae of aoe, f-t:ra-..."
8003,18530215,Advertisement | Classified Advertisement,"BUSINESS . SPRING SALES, 1853, POPrLAn AND POP..."
8465,18521231,Front Page/Cover Story,BllSl'ORY OF 1852. AND GENERAL HISTORY OF I}3E...
15682,18580529,Article | Feature,AFFAIRS IN FRANCE. Gossip in Paris-The Queen o...
17788,18550616,Advertisement | Classified Advertisement,"PUBLIC MEETINGS. OF THE WAR OP 1812,-The V Cor..."
...,...,...,...
191104,18580106,Article | Feature,International Copyright. We are glad to be tha...
191503,18551208,Article | Feature,"BY THE 'S , FOREIGN CORRESPONDENCE OF THE DAIL..."
192985,18520309,Article | Feature,WVho it the MlJodel ? It seems to be admitted ...
208750,18511013,Article | Feature,We publish this morning an interesting letter ...


In [9]:
#prints the 100 characters before and after each use of the word utopia and notes the document it was found in
for index, row in utopia_fifties.iterrows():
    text = row['raw_text']
    positions = [i for i in range(len(text)) if text.startswith('utopia', i)]
    for pos in positions:
        start = max(pos - 100, 0)
        end = min(pos + 100, len(text))
        context = text[start:end]
        print(f"Document ID: {row[0]}, Context: ...{context}...")


Document ID: 18521231, Context: ...should have but one object in his AdministratioD,-" to reconstitute in 1 rance, convulsed by so and utopian schemes, a peace based on conciliations f or mere of prim 1 ciples of authority, love for tn...
Document ID: 18580529, Context: ...the . of Algeria ; and it is anticipated that we will have as the result of his labors one of those utopias of government for which the Prince is famous. Since the cousin returned from his voyage to S...
Document ID: 18550831, Context: ...ll require. What the Tiars to gain by such . it is not easy to discover; the of the two armies is a utopia. ald she creation of a , if it were . Prince DoLoovo 's reply to Admiral DcN.. DAS, upon the ...
Document ID: 18550917, Context: ... be decreased, or eradicated altogether; but we imagine that none but the most unpractical-the most utopian theories on which the world may become a water drinker, and nothing else, will view with dis...
Document ID: 18590226, Context: ...Mr. BUCII

/tmp/ipykernel_7585/604010175.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Document ID: {row[0]}, Context: ...{context}...")


In [10]:

def preprocess_text_for_embeddings(text, min_word_length=2, remove_digits=True):
    """
    Preprocess historical OCR text for word embeddings
    
    Parameters:
    - text: raw text string
    - min_word_length: minimum word length to keep (default 2 to remove standalone chars)
    - remove_digits: whether to remove tokens that are purely numeric
    """
    if not isinstance(text, str):
        return ""
    
    # 1. Lowercase everything
    text = text.lower()
    
    # 2. Remove special characters but keep basic punctuation temporarily
    # This helps identify sentence boundaries
    # Keep: letters, numbers, spaces, basic punctuation (.,!?)
    text = re.sub(r'[^a-z0-9\s.,!?]', ' ', text)
    
    # 3. Normalize whitespace
    text = re.sub(r'\s+', ' ', text)
    
    # 4. Remove punctuation now that we've normalized
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # 5. Tokenize
    tokens = text.split()
    
    # 6. Filter tokens
    filtered_tokens = []
    for token in tokens:
        # Skip if too short
        if len(token) < min_word_length:
            continue
        
        # Skip if purely numeric 
        if remove_digits and token.isdigit():
            continue
        
        # Skip if mostly digits 
        # Count digit ratio
        digit_count = sum(c.isdigit() for c in token)
        if digit_count / len(token) > 0.3:  # More than 30% digits
            continue
        
        filtered_tokens.append(token)
    
    return ' '.join(filtered_tokens)


In [11]:
#make a new df from first 10k rows of fifties
fifties_sample = fifties.head(10000).copy()

In [12]:
from nltk.corpus import words
nltk.download('words')

[nltk_data] Downloading package words to /home/josh/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [13]:
english_words = set(words.words())

In [14]:
len(english_words)

235892

In [15]:
if "paris" in english_words:
    print("Found 'paris' in English words")

Can't use dictionary to filter OCR errors because it will filter out valid words that are not in the dictionary.

In [16]:
"""

def preprocess_dataframe(df, text_column='raw_text'):
    """
    #Apply preprocessing to entire dataframe
    """
    print("Preprocessing text...")
    print(f"Original articles: {len(df)}")
    
    # Apply preprocessing
    df['processed_text'] = df[text_column].apply(preprocess_text_for_embeddings)
    
    # Calculate statistics
    df['token_count'] = df['processed_text'].str.split().str.len()
    
    # Filter out articles with too few tokens (likely OCR failures or non-article content)
    min_tokens = 10
    df_filtered = df[df['token_count'] >= min_tokens].copy()
    
    print(f"After filtering (min {min_tokens} tokens): {len(df_filtered)}")
    print(f"Removed: {len(df) - len(df_filtered)} articles")
    print(f"\nToken statistics:")
    print(df_filtered['token_count'].describe())
    
    return df_filtered

"""

IndentationError: unexpected indent (1299947599.py, line 6)

In [ ]:
#preprocess_dataframe(fifties_sample)
#This is crashing my kernel due to memory issues, so I will not run it here.

Preprocessing text...
Original articles: 10000
